In [1]:
### Before we submit, review which modules we actually use !!!

%load_ext autoreload
%autoreload 2

import warnings
import calendar

import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.pylab as pltlab

from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

import pipeline as pipe
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')
%matplotlib inline
sns.set(rc={'figure.figsize':(11, 4)})

# Data Import & Exploration

In [2]:
data = pd.read_pickle('data/final_dataset.pk1')

In [3]:
pipe.find_outliers(data, 'median_income', 0, 10000000)

Number of outliers found: 0
Outlier values found: []


,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,log_med_income,prop_no_internet,prop_ba,prop_services,pop_density,FIPS,covid_cases,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change


In [4]:
data.head()

,name,total_pop,median_income,state,county,county_name,state_name,prop_white,prop_black,prop_hisp,log_med_income,prop_no_internet,prop_ba,prop_services,pop_density,FIPS,covid_cases,Testing_Rate,gov_party,election_diff,Apr-19,Mar-19,Feb-20,Mar-20,days_closed,yearly_change,monthly_change
0,"Washington County, Mississippi",47086,30834.0,28,151,Washington,Mississippi,0.256913,0.721701,0.015482,10.336373,0.336958,0.187937,0.100274,62.178441,28151.0,77,2291.374085,1,-0.364695,6.4,7.5,7.4,7.0,17.0,-0.066667,-0.054054
1,"Perry County, Mississippi",12028,39007.0,28,111,Perry,Mississippi,0.787745,0.196874,0.015048,10.571496,0.310103,0.109539,0.078850,18.434583,28111.0,27,2291.374085,1,0.536635,5.7,6.2,6.8,6.3,17.0,0.016129,-0.073529
2,"Choctaw County, Mississippi",8321,37203.0,28,019,Choctaw,Mississippi,0.676722,0.311982,0.003966,10.524145,0.368837,0.176582,0.014207,19.794546,28019.0,13,2291.374085,1,0.386224,4.8,5.3,5.4,5.0,17.0,-0.056604,-0.074074
3,"Itawamba County, Mississippi",23480,40510.0,28,057,Itawamba,Mississippi,0.909114,0.071593,0.015332,10.609304,0.226281,0.134468,0.057524,44.138916,28057.0,57,2291.374085,1,0.755161,4.1,4.4,4.7,4.7,17.0,0.068182,0.000000
4,"Carroll County, Mississippi",10129,43060.0,28,015,Carroll,Mississippi,0.643992,0.345839,0.002863,10.670350,0.332969,0.145006,0.057415,15.774563,28015.0,38,2291.374085,1,0.383321,5.5,5.8,6.2,6.0,17.0,0.034483,-0.032258


In [5]:
pipe.get_summary_stats(data)

          total_pop  median_income   prop_white   prop_black    prop_hisp  \
count  2.826000e+03    2825.000000  2826.000000  2826.000000  2826.000000   
mean   1.117590e+05   51714.241416     0.827329     0.098748     0.091962   
std    3.414925e+05   13821.224638     0.164131     0.149708     0.135249   
min    4.180000e+02   20188.000000     0.093534     0.000000     0.000000   
25%    1.412200e+04   42491.000000     0.757614     0.008110     0.021500   
50%    3.013550e+04   49936.000000     0.889050     0.028182     0.041568   
75%    7.725500e+04   57886.000000     0.947572     0.117748     0.095686   
max    1.009805e+07  136268.000000     0.997743     0.874123     0.990688   

       log_med_income  prop_no_internet      prop_ba  prop_services  \
count     2825.000000       2826.000000  2826.000000    2825.000000   
mean        10.821023          0.225514     0.217787       0.083571   
std          0.251809          0.086132     0.096556       0.034479   
min          9.912844 

After reviewing the summary statistics, we confirmed that the median income is non-negative. We also looked into the potential outlier on population density and confirmed that that value is New York City.

In [6]:
data.dtypes

name                 object
total_pop             int64
median_income       float64
state                object
county               object
county_name          object
state_name           object
prop_white          float64
prop_black          float64
prop_hisp           float64
log_med_income      float64
prop_no_internet    float64
prop_ba             float64
prop_services       float64
pop_density         float64
FIPS                float64
covid_cases           int64
Testing_Rate        float64
gov_party             int64
election_diff       float64
Apr-19              float64
Mar-19              float64
Feb-20              float64
Mar-20              float64
days_closed         float64
yearly_change       float64
monthly_change      float64
dtype: object

# Model Fitting & Evaluation

In [7]:
best_models = {}

## Monthly Data

### With States

In [8]:
# Select variables from full dataframe
df_mo_st = data.drop(['Apr-19','Mar-20','Feb-20','Mar-20','median_income',
                  'yearly_change','FIPS','name','state_name','county_name','county','total_pop'], axis=1)

In [9]:
# One-hot encode state
df_mo_st = pipe.hot_encode(df_mo_st, ['state'])

# Create training and testing sets
train, test = pipe.split_data(df_mo_st)

# Impute/nomralize continuous variables
numeric_cols = train.select_dtypes(include=['float64']).columns
train, test = pipe.impute_missing(train, test, numeric_cols)
train, test = pipe.normalize(train, test, numeric_cols)

# Separate features and targets
train_features = train.drop('monthly_change', axis=1)
test_features = test.drop('monthly_change', axis=1)

train_target = train['monthly_change']
test_target = test['monthly_change']

Training set contains 2260 observations
Testing set contains 566 observation

Imputing log_med_income missing values with median 10.819318178577827
Imputing prop_services missing values with median 0.0790869921304704


### Ridge

In [10]:
# Set up Pipeline for Ridge
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge(alpha=0.1))
])

alpha_range = np.arange(0,1,0.1)
params = {
          'ridge__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   55.8s finished


,param_ridge__alpha,param_poly__degree,rank_test_score,mean_test_score
1,0.1,1,1,-1.685449e-01
2,0.2,1,2,-1.686022e-01
3,0.3,1,3,-1.687339e-01
4,0.4,1,4,-1.689261e-01
5,0.5,1,5,-1.691684e-01
6,0.6,1,6,-1.694530e-01
7,0.7,1,7,-1.697736e-01
8,0.8,1,8,-1.701252e-01
9,0.9,1,9,-1.705039e-01
11,0.1,2,10,-1.901613e+00


In [11]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['state_ridge'] = d

# Rate coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('ridge', Ridge(alpha=0.1))]) 

Mean Squared Error: 0.18736064414266587
R-Squared: 0.8166495475125528
Adjusted R-Squared: 0.7928139886891847


,feature,coefficient,coefficient (absolute)
19,state_08,2.6923,2.692297
42,state_32,2.44908,2.449083
25,state_15,-1.75099,1.750993
32,state_22,1.6689,1.668903
41,state_31,1.62984,1.629839
...,...,...,...
7,pop_density,-0.00644039,0.006440
13,days_closed,-0.00519993,0.005200
2,prop_hisp,-0.00408755,0.004088
8,covid_cases,6.83214e-06,0.000007


### Lasso

In [12]:
# Lasso
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('lasso', Lasso(alpha=0.1))
])

params = {
          'lasso__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.0min finished


,param_lasso__alpha,param_poly__degree,rank_test_score,mean_test_score
0,0,1,1,-0.168597
2,0.1,1,2,-0.935472
4,0.2,1,3,-0.990357
18,0.9,1,4,-1.009132
16,0.8,1,5,-1.009151
14,0.7,1,6,-1.009170
12,0.6,1,7,-1.009188
10,0.5,1,8,-1.009207
8,0.4,1,9,-1.009225
6,0.3,1,10,-1.009244


In [13]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['state_lasso'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('lasso', Lasso(alpha=0.0))]) 

Mean Squared Error: 0.18731134203149158
R-Squared: 0.8166977943812255
Adjusted R-Squared: 0.7928685076507849


,feature,coefficient,coefficient (absolute)
19,state_08,2.6046,2.604602
42,state_32,2.39083,2.390833
32,state_22,1.88689,1.886891
25,state_15,-1.78918,1.789180
41,state_31,1.50988,1.509880
...,...,...,...
7,pop_density,-0.00688236,0.006882
2,prop_hisp,-0.00600833,0.006008
13,days_closed,-0.00302038,0.003020
8,covid_cases,6.95327e-06,0.000007


### Linear Regression

In [14]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('linreg', LinearRegression())
])

params = {
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    6.7s finished


,param_poly__degree,rank_test_score,mean_test_score
0,1,1,-0.168581
1,2,2,-153.970969


In [15]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['state_linear'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('linreg', LinearRegression())]) 

Mean Squared Error: 0.1874139411701096
R-Squared: 0.8165973912331871
Adjusted R-Squared: 0.7927550520935014


,feature,coefficient,coefficient (absolute)
19,state_08,2.7037,2.703703e+00
42,state_32,2.4709,2.470902e+00
25,state_15,-1.81576,1.815757e+00
32,state_22,1.67465,1.674654e+00
41,state_31,1.63888,1.638880e+00
...,...,...,...
2,prop_hisp,-0.00600833,6.008327e-03
10,gov_party,-0.00399393,3.993926e-03
13,days_closed,-0.00302038,3.020378e-03
8,covid_cases,6.95327e-06,6.953271e-06


### Elastic Net

In [16]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('en',ElasticNet())
])

params = {'en__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished


,param_en__alpha,param_poly__degree,rank_test_score,mean_test_score
0,0,1,1,-0.168597
2,0.1,1,2,-0.907762
4,0.2,1,3,-0.938950
6,0.3,1,4,-0.967572
8,0.4,1,5,-0.993248
10,0.5,1,6,-1.008761
18,0.9,1,7,-1.009216
16,0.8,1,8,-1.009225
14,0.7,1,9,-1.009235
12,0.6,1,10,-1.009244


In [17]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['state_en'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('en', ElasticNet(alpha=0.0))]) 

Mean Squared Error: 0.18731134203149158
R-Squared: 0.8166977943812255
Adjusted R-Squared: 0.7928685076507849


,feature,coefficient,coefficient (absolute)
19,state_08,2.6046,2.604602
42,state_32,2.39083,2.390833
32,state_22,1.88689,1.886891
25,state_15,-1.78918,1.789180
41,state_31,1.50988,1.509880
...,...,...,...
7,pop_density,-0.00688236,0.006882
2,prop_hisp,-0.00600833,0.006008
13,days_closed,-0.00302038,0.003020
8,covid_cases,6.95327e-06,0.000007


### Without States

In [18]:
# Select variables from full dataframe
df_mo = data.drop(['Apr-19','Mar-20','Feb-20','Mar-20','median_income',
                  'yearly_change','FIPS','name','state_name','county_name','county','state','total_pop'], axis=1)

In [19]:
# Create training and testing sets
train, test = pipe.split_data(df_mo)

# Impute/normalize continuous variables
numeric_cols = train.select_dtypes(include=['float64']).columns #Normalizes state one-hots as well
train, test = pipe.impute_missing(train, test, numeric_cols)
train, test = pipe.normalize(train, test, numeric_cols)

# Separate features and targets                              
train_features = train.drop('monthly_change', axis=1)
test_features = test.drop('monthly_change', axis=1)

train_target = train['monthly_change']
test_target = test['monthly_change']

Training set contains 2260 observations
Testing set contains 566 observation

Imputing log_med_income missing values with median 10.819318178577827
Imputing prop_services missing values with median 0.0790869921304704


### Ridge

In [20]:
# Set up Pipeline for Ridge
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('ridge', Ridge(alpha=0.1))
])

alpha_range = np.arange(0,1,0.1)
params = {
          'ridge__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished


,param_ridge__alpha,param_poly__degree,rank_test_score,mean_test_score
9,0.9,1,1,-0.846575
8,0.8,1,2,-0.846576
7,0.7,1,3,-0.846578
6,0.6,1,4,-0.846580
5,0.5,1,5,-0.846582
4,0.4,1,6,-0.846584
3,0.3,1,7,-0.846586
2,0.2,1,8,-0.846588
1,0.1,1,9,-0.846590
0,0,1,10,-0.846592


In [21]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['ridge'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('ridge', Ridge(alpha=0.9))]) 

Mean Squared Error: 0.7921055929747043
R-Squared: 0.22484831564113306
Adjusted R-Squared: 0.205152991537641


,feature,coefficient,coefficient (absolute)
1,prop_black,0.391357,0.391357
2,prop_hisp,0.345247,0.345247
11,election_diff,0.253586,0.253586
5,prop_ba,0.188023,0.188023
0,prop_white,0.187301,0.187301
10,gov_party,-0.182176,0.182176
12,Mar-19,-0.155789,0.155789
6,prop_services,0.11172,0.111720
3,log_med_income,-0.0954635,0.095463
9,Testing_Rate,-0.0945238,0.094524


### Lasso

In [22]:
# Lasso
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('lasso', Lasso(alpha=0.1))
])

params = {
          'lasso__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.4s finished


,param_lasso__alpha,param_poly__degree,rank_test_score,mean_test_score
0,0,1,1,-0.846592
2,0.1,1,2,-0.935472
4,0.2,1,3,-0.990357
18,0.9,1,4,-1.009132
16,0.8,1,5,-1.009151
14,0.7,1,6,-1.009170
12,0.6,1,7,-1.009188
10,0.5,1,8,-1.009207
8,0.4,1,9,-1.009225
6,0.3,1,10,-1.009244


In [23]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['lasso'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('lasso', Lasso(alpha=0.0))]) 

Mean Squared Error: 0.7920966182783529
R-Squared: 0.22485709824670363
Adjusted R-Squared: 0.2051619972947143


,feature,coefficient,coefficient (absolute)
1,prop_black,0.392702,0.392702
2,prop_hisp,0.345741,0.345741
11,election_diff,0.254156,0.254156
5,prop_ba,0.188521,0.188521
0,prop_white,0.188299,0.188299
10,gov_party,-0.182851,0.182851
12,Mar-19,-0.155783,0.155783
6,prop_services,0.11185,0.111850
3,log_med_income,-0.0955839,0.095584
9,Testing_Rate,-0.0945105,0.094510


### Linear Regression

In [24]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('linreg', LinearRegression())
])

params = {
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished


,param_poly__degree,rank_test_score,mean_test_score
0,1,1,-0.846592
1,2,2,-3.703404


In [25]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['linear'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('linreg', LinearRegression())]) 

Mean Squared Error: 0.7920966182783583
R-Squared: 0.2248570982466983
Adjusted R-Squared: 0.20516199729470885


,feature,coefficient,coefficient (absolute)
1,prop_black,0.392702,0.392702
2,prop_hisp,0.345741,0.345741
11,election_diff,0.254156,0.254156
5,prop_ba,0.188521,0.188521
0,prop_white,0.188299,0.188299
10,gov_party,-0.182851,0.182851
12,Mar-19,-0.155783,0.155783
6,prop_services,0.11185,0.111850
3,log_med_income,-0.0955839,0.095584
9,Testing_Rate,-0.0945105,0.094510


### Elastic Net

In [26]:
pipeline = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)), ('en',ElasticNet())
])

params = {'en__alpha': alpha_range,
          'poly__degree': (1,2)
         }

grid_model, grid_model_result, cv_results = pipe.run_gridsearch(pipeline, params, train_features, train_target, verbose=1)

cv_results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.1s finished


,param_en__alpha,param_poly__degree,rank_test_score,mean_test_score
0,0,1,1,-0.846592
2,0.1,1,2,-0.909084
4,0.2,1,3,-0.938950
6,0.3,1,4,-0.967572
8,0.4,1,5,-0.993248
10,0.5,1,6,-1.008761
18,0.9,1,7,-1.009216
16,0.8,1,8,-1.009225
14,0.7,1,9,-1.009235
12,0.6,1,10,-1.009244


In [27]:
print(grid_model_result.best_estimator_, '\n')

# Evaluate model
pipe.evaluate_model(grid_model, test_features, test_target)

mse, r2, adj = pipe.evaluate_model(grid_model, test_features, test_target)

d = {'model': grid_model_result,
     'mse': mse,
     'r2': r2,
     'adj': adj}

best_models['en'] = d

# Rank coefficients
pipe.rank_coefs(grid_model_result, train_features.columns)

Pipeline(steps=[('poly', PolynomialFeatures(degree=1, include_bias=False)),
                ('en', ElasticNet(alpha=0.0))]) 

Mean Squared Error: 0.7920966182783529
R-Squared: 0.22485709824670363
Adjusted R-Squared: 0.2051619972947143
Mean Squared Error: 0.7920966182783529
R-Squared: 0.22485709824670363
Adjusted R-Squared: 0.2051619972947143


,feature,coefficient,coefficient (absolute)
1,prop_black,0.392702,0.392702
2,prop_hisp,0.345741,0.345741
11,election_diff,0.254156,0.254156
5,prop_ba,0.188521,0.188521
0,prop_white,0.188299,0.188299
10,gov_party,-0.182851,0.182851
12,Mar-19,-0.155783,0.155783
6,prop_services,0.11185,0.111850
3,log_med_income,-0.0955839,0.095584
9,Testing_Rate,-0.0945105,0.094510


In [28]:
best = pd.DataFrame.from_dict(best_models).transpose()
best.columns = ['Model Objects', 'Mean Squared Error', 'R-Squared', 'Adjusted R-Squared']
best.sort_values(by='Adjusted R-Squared',ascending=False)


,Model Objects,Mean Squared Error,R-Squared,Adjusted R-Squared
state_lasso,"GridSearchCV(cv=5,\n estimator=Pip...",0.187311,0.816698,0.792869
state_en,"GridSearchCV(cv=5,\n estimator=Pip...",0.187311,0.816698,0.792869
state_ridge,"GridSearchCV(cv=5,\n estimator=Pip...",0.187361,0.81665,0.792814
state_linear,"GridSearchCV(cv=5,\n estimator=Pip...",0.187414,0.816597,0.792755
lasso,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
en,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
linear,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
ridge,"GridSearchCV(cv=5,\n estimator=Pip...",0.792106,0.224848,0.205153


In [29]:
best.sort_values(by='Mean Squared Error')

,Model Objects,Mean Squared Error,R-Squared,Adjusted R-Squared
state_lasso,"GridSearchCV(cv=5,\n estimator=Pip...",0.187311,0.816698,0.792869
state_en,"GridSearchCV(cv=5,\n estimator=Pip...",0.187311,0.816698,0.792869
state_ridge,"GridSearchCV(cv=5,\n estimator=Pip...",0.187361,0.81665,0.792814
state_linear,"GridSearchCV(cv=5,\n estimator=Pip...",0.187414,0.816597,0.792755
lasso,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
en,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
linear,"GridSearchCV(cv=5,\n estimator=Pip...",0.792097,0.224857,0.205162
ridge,"GridSearchCV(cv=5,\n estimator=Pip...",0.792106,0.224848,0.205153
